In [ ]:
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

import img_seg_package.single_image as si

bokeh.io.output_notebook()

The purpose of this notebook is to provide a method for generating the associated figures of the CAP-A4 manuscript.

## Figure 2:

First, we want to generate an overview of the lung. To do this, we can load in an image from...

**Make an overview of the lung for Figure 2**

Next, we want to generate all the individual figures to demonstrate the workflow of the figure. To do this, we need to identify the location of the specific tissue image we want to examine. 

In [ ]:
# The overall directory for the examined images
data_dir = '../Lung_Paper_Images/Test_Images/'

# The tissue examined for filename purposes
tissue = 'Lung'

# The variant being examined
variant = 'CAPA4'

# The animal being examined (integer for dataframe purposes)
animal = 2

# The replicate being examined (integer for dataframe purposes)
replicate = 2

# The name of the image being examined
img_loc = '1_XY04_00065_Overlay'

# The complete filename of the image being examined 
fname = data_dir + tissue + '/' + variant + '/' + str(animal) + '/' + str(replicate) + '/' + img_loc + '.tif'

Next, we can load in the dataframe in which the data has been saved. From this dataframe we can extract all of the parameters used to analyze the whole tissue images and apply them to the individual images. 

In [ ]:
# Indicate the dataframe location, where it was previously saved it using the 'image_analysis.ipynb' notebook.
dataframe_loc = '../Lung_Paper_Images/Quantification/Lung_Quantification.csv'

# Create the dataframe
df = pd.read_csv(dataframe_loc)

# Set indices of the different identifying information to allow simple access of the data.
inds = (df['Virus'] == variant) & (df['Animal'] == animal) & (df['Replicate'] == replicate)

Now that we have all of this information initialized, we can go through the analysis process to view the outcome of the major analysis steps. 

In [ ]:
#Load in the autofluorescence and signal images
im_af = skimage.img_as_float(skimage.io.imread(fname)[:,:,0])
im_sig = skimage.img_as_float(skimage.io.imread(fname)[:,:,1])

#Get the image height and width data
im_height = np.shape(im_sig)[0]
im_width = np.shape(im_sig)[1]

#Import the shape template and get the associated data
template = np.load('../template.npy')
template_height = np.shape(template)[0]
template_width = np.shape(template)[1]

#Save these initial images for the figure
skimage.io.imsave('../Workflow_Figure/autofluorescence_input.tif', im_af[300:800, 450:950])
skimage.io.imsave('../Workflow_Figure/signal_input.tif', im_sig[300:800, 450:950])

#Show these images
bokeh.io.show(si.show_two_ims(im_af[300:800, 450:950], im_sig[300:800, 450:950])) #, color_mapper='rgb'))

Next, we can display the image with the two channels stacked. 

In [ ]:
# Make a stack of the three channels to create a magenta / green image, with labelled cells in green.
im_magenta = np.dstack((im_af, im_sig, im_af))

# Display the stacked image
bokeh.io.show(si.imshow(im_magenta[300:800, 450:950], color_mapper='rgb'))

Now that we have our images loaded, we can start to perform the analysis processes on these images. The first thing we can do is to determine the tissue area within the examined tissue. 

In [ ]:
# Perform the intensity threshold determined to dilineate background from tissue.
threshed_single = im_af > df.loc[inds, 'Area Threshold'].values[0]
skimage.io.imsave('../Workflow_Figure/tissue_area.tif', threshed_single[300:800, 450:950])

bokeh.io.show(si.imshow(threshed_single[300:800, 450:950])) #, color_mapper='rgb'))

In [ ]:
# Next, we can create the autofluorescence subtracted image
im_sub = im_sig - im_af*df.loc[inds, 'Image Multiplication Factor'].values[0]

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/af_subtracted_image.tif', im_sub[300:800, 450:950])

# Show the image
bokeh.io.show(si.imshow(im_sub[300:800, 450:950])) #, color_mapper='rgb'))

We can see that subtraction does a really good job illuminating where the expressing cells are located. Next, we can examine the background subtraction.

In [ ]:
# Next, we can perform background subtraction
im_bg = skimage.filters.gaussian(im_sub, df.loc[inds, 'Gaussian Size'].values[0], truncate = df.loc[inds, 'Truncation'].values[0])

im_no_bg = im_sub - im_bg

im_no_bg = (im_no_bg - df.loc[inds, 'Minimum Pixel Value'].values[0]) / (df.loc[inds, 'Maximum Pixel Value'].values[0] - df.loc[inds, 'Minimum Pixel Value'].values[0])

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/bg_subtracted_image.tif', im_no_bg[300:800, 450:950])

# Show the image
bokeh.io.show(si.imshow(im_no_bg[300:800, 450:950])) #, color_mapper='rgb'))

This image is a little smoother, and should be easier to identify the bright sections in. Next, we can do the feature matching on the subtracted image. 

In [ ]:
#Perform feature matching
im_feat = skimage.feature.match_template(im_sub, template)

#Display objects that pass the similarity threshold
binary_thresh = im_feat > df.loc[inds, 'Size Threshold'].values[0]

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/feature_thresholding.tif', binary_thresh[300:800, 450:950])

# Show the image
bokeh.io.show(si.imshow(binary_thresh[300:800, 450:950])) #, color_mapper='rgb'))

These are all the positions that have features similar to our template. Next we want to determine which regions of the image are bright. 

In [ ]:
# Determine the regions that are bright using the same intensity threhold as the large image. 
binary_intensity = im_no_bg[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))] > df.loc[inds, 'Applied Threshold'].values[0]

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/intensity_thresholding.tif', binary_intensity[300:800, 450:950])

# Show the image
bokeh.io.show(si.imshow(binary_intensity[300:800, 450:950])) #, color_mapper='rgb'))

These are all the bright regions in the image. There is definitely co-localization between these binary images, so we can multiply them together to determine regions that are both circular and bright. 

In [ ]:
binary_mult = binary_intensity*binary_thresh

binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=df.loc[inds, 'Minimum Size'].values[0])

im_labeled, n_labels = skimage.measure.label(binary_rem, background=0, return_num=True)

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/intensity_thresholding.tif', im_labeled[300:800, 450:950])

# Load phase image
im_p = skimage.img_as_float(skimage.io.imread(fname))[:,:,:2][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]

binary_rgb = np.dstack((binary_rem, binary_rem, binary_rem))

#Save this image for the figure
skimage.io.imsave('../Workflow_Figure/labeled_image.tif', binary_rgb[300:800, 450:950])

# Show the image
bokeh.io.show(si.imshow(binary_rgb[300:800, 450:950], color_mapper='rgb'))

These are all the spots that pass both of our tests.